In [1]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
#load data
data = pd.read_csv('merged_data.csv', encoding='utf-8')

In [3]:
data.head()

,stn_id,borrowed_hour,borrowed_day,is_holiday,borrowed_num_nearby,강수량(mm),wind_chill,stn_gu,nearby_id,borrowed_date,borrowed_num
0,ST-10,0,6,1,2,0.0,21.031237,마포구,ST-2167,20221001,2
1,ST-10,1,6,1,1,0.0,20.222173,마포구,ST-2167,20221001,2
2,ST-10,2,6,1,2,0.0,19.589061,마포구,ST-2167,20221001,1
3,ST-10,3,6,1,0,0.0,19.552158,마포구,ST-2167,20221001,0
4,ST-10,4,6,1,2,0.0,18.783459,마포구,ST-2167,20221001,0


# Total Data

In [8]:
# 필요한 열 선택
selected_features = ['stn_id', 'borrowed_hour', 'borrowed_day', 'is_holiday', 'borrowed_num_nearby', '강수량(mm)', 'wind_chill', 'stn_gu', 'nearby_id', 'borrowed_date', 'borrowed_num']
data = data[selected_features]

In [9]:
# 범주형 데이터를 숫자로 변환 (Label Encoding)
categorical_features = ['stn_id', 'stn_gu', 'nearby_id']
for feature in categorical_features:
    data[feature] = pd.factorize(data[feature])[0]

In [6]:
# 날짜 데이터 전처리
data['borrowed_date'] = pd.to_datetime(data['borrowed_date'])
data['year'] = data['borrowed_date'].dt.year
data['month'] = data['borrowed_date'].dt.month
data['day'] = data['borrowed_date'].dt.day

In [10]:
# 독립변수와 종속변수 분리
X = data.drop(['borrowed_num', 'stn_id', 'borrowed_date'], axis=1)
y = data['borrowed_num']

In [11]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Hyper Parameter Optimization

In [4]:
# LightGBM 모델
model = lgb.LGBMRegressor()

In [5]:
# Grid Search
param_grid = {
    'num_leaves': [20, 30, 40, 50, 60, 70, 80, 90, 100],
    'learning_rate': [0.01, 0.03, 0.04, 0.05, 0.1],
    'n_estimators': [1000, 10000]
}

In [11]:
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=3)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best RMSE:", (-grid_search.best_score_)**0.5)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.302542 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 5145300, number of used features: 11
[LightGBM] [Info] Start training from score 1.482044
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.256884 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 734
[LightGBM] [Info] Number of data points in the train set: 5145300, number of used features: 11
[LightGBM] [Info] Start training from score 1.484790
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.316353 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not eno

KeyboardInterrupt: 

In [12]:
# Random Search
param_dist = {
    'num_leaves': [30, 40, 50, 60, 70, 80, 90, 100],
    'learning_rate': [0.05],
    'n_estimators': [500, 1000, 5000, 10000]
}

In [13]:
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, scoring='neg_mean_squared_error', cv=3)
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
print("Best RMSE:", (-random_search.best_score_)**0.5)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.222869 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 727
[LightGBM] [Info] Number of data points in the train set: 5145300, number of used features: 8
[LightGBM] [Info] Start training from score 1.482044
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.283957 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 728
[LightGBM] [Info] Number of data points in the train set: 5145300, number of used features: 8
[LightGBM] [Info] Start training from score 1.484790
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.380014 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug

KeyboardInterrupt: 

## LightGBM (total)

In [131]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 80,
    'learning_rate': 0.03,
    'feature_fraction': 0.9
}


In [151]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'dart',
    'num_leaves': 100,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

## 교차검증

In [132]:
# K-fold 교차 검증 수행
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = lgb.cv(params, lgb.Dataset(X, label=y), num_boost_round=1000, folds=kf, callbacks=[
        lgb.early_stopping(stopping_rounds=10, verbose=50),
    ])


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.326699 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 737
[LightGBM] [Info] Number of data points in the train set: 7717950, number of used features: 11
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.282034 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 737
[LightGBM] [Info] Number of data points in the train set: 7717950, number of used features: 11
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.262550 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 737
[LightGBM] [Info] Number of data 

In [133]:
cv_results.keys()

dict_keys(['valid rmse-mean', 'valid rmse-stdv'])

In [134]:
# 교차 검증 결과 출력
print(f'Best number of boosting rounds: {len(cv_results["valid rmse-mean"])}')
print(f'Best RMSE: {cv_results["valid rmse-mean"][-1]}')

Best number of boosting rounds: 8905
Best RMSE: 1.948405555438897


In [152]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)


In [153]:
model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[test_data, train_data], callbacks=[
        lgb.early_stopping(stopping_rounds=3, verbose=100),
    ])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.344168 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 735
[LightGBM] [Info] Number of data points in the train set: 7717950, number of used features: 11
[LightGBM] [Info] Start training from score 1.483498


c:\ProgramData\anaconda3\Lib\site-packages\lightgbm\callback.py:292: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


In [155]:
model = lgb.train(params, train_data, num_boost_round=100000, valid_sets=[test_data, train_data], callbacks=[
        lgb.early_stopping(stopping_rounds=10, verbose=50),
    ])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.416849 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 735
[LightGBM] [Info] Number of data points in the train set: 7717950, number of used features: 11
[LightGBM] [Info] Start training from score 1.483498


KeyboardInterrupt: 

In [156]:
# 모델 평가
y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)
y_pred_test = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
# 훈련 데이터에서의 평가
mse_train = mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)

print(f'Training MSE: {mse_train}')
print(f'Training MAE: {mae_train}')
print(f'Training RMSE: {rmse_train}')

# 검증 데이터에서의 평가
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)

print(f'Test MSE: {mse_test}')
print(f'Test MAE: {mae_test}')
print(f'Test RMSE: {rmse_test}')

Training MSE: 3.737101203294012
Training MAE: 1.1228728212762518
Training RMSE: 1.9331583492549211
Test MSE: 3.9273722893651293
Test MAE: 1.1337698955516267
Test RMSE: 1.9817598970019374


In [ ]:
print(f'Training - Test MSE: {mse_train - mse_test}')
print(f'Training - Test MAE: {mae_train - mae_test}')
print(f'Training - Test RMSE: {rmse_train - rmse_test}')

Training - Test MSE: -0.19027108607111742
Training - Test MAE: -0.010897074275374896
Training - Test RMSE: -0.048601547747016305


In [ ]:
# 모델 평가
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')

# Sample Data

In [17]:
# 필요한 열 선택
selected_features = ['stn_id', 'borrowed_hour', 'borrowed_day', 'is_holiday', 'borrowed_num_nearby', '강수량(mm)', 'wind_chill', 'stn_gu', 'nearby_id', 'borrowed_date', 'borrowed_num']
data = data[selected_features]

In [18]:
# 범주형 데이터를 숫자로 변환 (Label Encoding)
categorical_features = ['stn_id', 'stn_gu', 'nearby_id']
for feature in categorical_features:
    data[feature] = pd.factorize(data[feature])[0]

In [19]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(data.drop('borrowed_num', axis=1), data['borrowed_num'], test_size=0.2, random_state=42)

In [20]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

In [21]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [25]:
model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[test_data], callbacks=[
        lgb.early_stopping(stopping_rounds=3, verbose=100),
    ])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.322341 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1136
[LightGBM] [Info] Number of data points in the train set: 7717950, number of used features: 10
[LightGBM] [Info] Start training from score 1.483498
Training until validation scores don't improve for 3 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 1.98518


In [26]:
# 모델 평가
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# MSE 계산
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse}')

# MAE 계산
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

# RMSE 계산
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error (RMSE): {rmse}')

Mean Squared Error (MSE): 3.940949911399397
Mean Absolute Error (MAE): 1.1485539408033907
Root Mean Squared Error (RMSE): 1.985182588932161


# Overfitting Check

In [4]:
# 필요한 열 선택
selected_features = ['stn_id', 'borrowed_hour', 'borrowed_day', 'is_holiday', 'borrowed_num_nearby', '강수량(mm)', 'wind_chill', 'stn_gu', 'nearby_id', 'borrowed_date', 'borrowed_num']
data = data[selected_features]


In [6]:
# 범주형 데이터를 숫자로 변환 (Label Encoding)
categorical_features = ['stn_id', 'stn_gu', 'nearby_id']
for feature in categorical_features:
    data[feature] = pd.factorize(data[feature])[0]


In [ ]:
# 날짜 데이터 전처리
data['borrowed_date'] = pd.to_datetime(data['borrowed_date'])
data['year'] = data['borrowed_date'].dt.year
data['month'] = data['borrowed_date'].dt.month
data['day'] = data['borrowed_date'].dt.day

In [7]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(data.drop('borrowed_num', axis=1), data['borrowed_num'], test_size=0.2, random_state=42)


In [30]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}


### Change Learning Rate: 0.01

In [13]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 50,
    'learning_rate': 0.01,
    'feature_fraction': 0.9
}

### Change Learning Rate: 0.001

In [18]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 50,
    'learning_rate': 0.001,
    'feature_fraction': 0.9
}

### Chage Learning Rate: 0.05 + leaves: 50

In [33]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 70,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

### Change Learning Rate: 0.1

In [38]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 70,
    'learning_rate': 0.1,
    'feature_fraction': 0.9
}

In [39]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)


In [40]:
model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[test_data, train_data], callbacks=[
        lgb.early_stopping(stopping_rounds=3, verbose=100),
    ])


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.309523 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1136
[LightGBM] [Info] Number of data points in the train set: 7717950, number of used features: 10
[LightGBM] [Info] Start training from score 1.483498
Training until validation scores don't improve for 3 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 1.7775	valid_0's rmse: 1.81438


In [41]:
# 모델 평가
y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)
y_pred_test = model.predict(X_test, num_iteration=model.best_iteration)


In [42]:
# 훈련 데이터에서의 평가
mse_train = mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)


print(f'Training MSE: {mse_train}')
print(f'Training MAE: {mae_train}')
print(f'Training RMSE: {rmse_train}')


# 검증 데이터에서의 평가
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)

print(f'Test MSE: {mse_test}')
print(f'Test MAE: {mae_test}')
print(f'Test RMSE: {rmse_test}')

Training MSE: 3.1595104943019185
Training MAE: 1.0583968138237099
Training RMSE: 1.7775011938960599
Test MSE: 3.2919886643182226
Test MAE: 1.0675021941315082
Test RMSE: 1.8143838249715032
